# 실시간 프로그램 클론코딩

In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import datetime

In [ ]:
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(300, 300, 400, 300)
        self.setWindowTitle("실시간 프로그램")
        
        self.range = None
        self.target = None
        self.account = None
        self.amount = None
        self.hold = None
        
        self.previous_day_hold = False
        self.previous_day_quantity = False
        
        self.plain_text_edit = QPlainTextEdit(self)
        self.plain_text_edit.setReadOnly(True)
        self.plain_text_edit.move(10, 10)
        self.plain_text_edit.resize(380, 280)
        
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self._handler_login)
        self.ocx.OnReceiveTrData.connect(self._handler_tr_data)
        self.ocx.OnReceiveRealData.connect(self._handler_real_data)
        self.ocx.OnReceiveChejanData.connect(self._handler_chejan_data)
        
        self.login_event_loop = QEventLoop()
        self.CommConnect()
        self.run()
        
    def CommConnect(self):
        self.ocx.dynamicCall("CommConnect()")
        self.login_event_loop.exec()
        
    def run(self):
        accounts = self.GetLoginInfo("ACCNO")
        self.account = accounts.split(";")[0]
        print(self.account)
        
        # TR 요청
        self.request_opt10081()
        self.request_opw00001()
        self.request_opw00004()
        
        # 실시간 주식체결
        self.subscribe_market_time("1")
        self.subscribe_stock_conclusion("2")
        
    def GetLoginInfo(self, tag):
        data = self.ocx.dynamicCall("GetLoginInfo(QString)", tag)
        return data
    
    def _handler_login(self, err_code):
        if err_code == 0:
            self.plain_text_edit.appendPlainText("로그인 완료")
        self.login_event_loop.exit()
        
    def _handler_tr_data(self, screen_no, rqname, trcode, record, next):
        if rqname == "KODEX일봉데이터":
            now = datetime.datetime.now()
            today = now.strftime("%Y%m%d")
            date = self.GetCommData(trcode, rqname, 0, "일자")
            
            if date != today:
                high = self.GetCommData(trcode, rqname, 0, "고가")
                low = self.GetCommData(trcode, rqname, 0, "저가")
            else:
                date = self.GetCommData(trcode, rqname, 1, "일자")
                high = self.GetCommData(trcode, rqname, 1, "고가")
                low = self.GetCommData(trcode, rqname, 1, "저가")
                
            self.range = int(high) - int(low)
            info = f"일자: {date} 고가: {high} 저가: {low} 전일변동: {self.range}"
            self.plain_text_edit.appendPlainText(info)
        
        elif rqname == "예수금조회":
            orderable_amount = self.GetCommData(trcode, rqname, 0, "주문가능금액")
            orderable_amount = int(orderable_amount)
            self.amount = int(orderable_amount * 0.2)
            self.plain_text_edit.appendPlainText(f"주문가능금액: {orderable_amount} 투자금액: {self.amount}")
            
        elif rqname == "계좌평가현황":
            rows = self.GetRepeatCnt(trcode, rqname)
            for i in range(rows):
                code = self.GetCommData(trcode, rqname, i, "종목코드")
                quantity = self.GetCommData(trcode, rqname, i, "보유수량")

                if code[1:] == "229200":
                    self.previous_day_hold = True
                    self.previous_day_quantity = int(quantity)
    
    def GetRepeatCnt(self, trcode, rqname):
        ret = self.ocx.dynamicCall("GetRepeatCnt(QString, QString)", trcode, rqname)
        return ret
    
    def request_opt10081(self):
        now = datetime.datetime.now()
        today = now.strftime("%Y%m%d")
        self.SetInputValue("종목코드", "229200")
        self.SetInputValue("기준일자", today)
        self.SetInputValue("수정주가구분", 1)
        self.CommRqData("KODEX일봉데이터", "opt10081", 0, "9000")
        
    def request_opw00001(self):
        self.SetInputValue("계좌번호", self.account)
        self.SetInputValue("비밀번호", "")
        self.SetInputValue("비밀번호입력매체구분", "00")
        self.SetInputValue("조회구분", 2)
        self.CommRqData("예수금조회", "opw00001", 0, "9001")
        
    def request_opw00004(self):
        self.SetInputValue("계좌번호", self.account)
        self.SetInputValue("비밀번호", "")
        self.SetInputValue("상장폐지조회구분", 0)
        self.SetInputValue("비밀번호입력매체구분", "00")
        self.SetInputValue("계좌평가현황", "opw00004", 0, "9002")
        
    def SetRealReg(self, screen_no, code_list, fid_list, real_type):
        self.ocx.dynamicCall("SetRealReg(QString, QString, QString, QString)", screen_no, code_list, fid_list, real_type)
        
    def GetCommRealData(self, code, fid):
        data = self.ocx.dynamicCall("GetCommRealData(QString, int)", code, fid)
        return data
    
    def DisConnectRealData(self, screen_no):
        self.ocx.dynamicCall("DisConnectRealData(QString)", screen_no)
        
    def _handler_real_data(self, code, real_type, real_data):
        if real_type == "장시작시간":
            state = self.GetCommRealData(code, 215)
            if state == "3":
                if self.previous_day_hold:
                    self.previous_day_hold = False
                    # 시장가 매도
                    self.SendOrder("매도", "8001", self.account, 2, "229200", self.previous_day_quantity, 0, "03", "")
                
            elif state == "4":
                QCoreApplication.instance().quit()
                print("메인 윈도우 종료")
                
        elif real_type == "주식체결":
            current_price = self.GetCommRealData(code, 10)
            current_prcie = abs(int(current_price))
            conclusion_time = self.GetCommRealData(code, 20)
            
            if self.range is not None and self.target is None:
                open_price = self.GetCommRealData(code, 16)
                open_price = abs(int(open_price))
                self.target = int(open_price + (self.range * 0.5))
                self.plain_text_edit.appendPlainText(f"목표가 계산: {self.target}")
                
            if self.hold is None and self.target is not None and self.amount is not None and current_prcie > self.target:
                self.hold = True
                quantity = int(self.amount / current_prcie)
                self.SendOrder("매수", "8000", self.account, 1, "229200", quantity, 0, "03", "")
                self.plain_text_edit.appendPlainText(f"시장가 매수 진행 수량: {quantity}")
                
            self.plain_text_edit.appendPlainText(f"시간: {conclusion_time} 현재가: {current_prcie} 보유여부: {self.hold}")
            
                
        
        